In [20]:
import gdal
import glob
import multiprocessing as mp
from wsmtk.utils import *
import re
from contextlib import contextmanager, closing
import matplotlib.pyplot as plt
import time

In [3]:
def init_parameters(**kwargs):
    '''Initialize parameters for processing in workers.'''

    params = dict()

    for key, value in kwargs.items():
        params[key] = value
    return params


def init(arr_,dim,files_):
    
    global arr
    global files
    
    files = files_
    arr = tonumpyarray(arr_)
    arr.shape = dim
    
def wload(ix):
    
    xoff, yoff, xsize, ysize, fix = ix
    
    ds = gdal.Open(files[fix])
    sds_o = gdal.Open(ds.GetSubDatasets()[0][0])
    
    
    arr[...,fix] =  sds_o.ReadAsArray(xoff,yoff,xsize,ysize).flatten()
    ds = None
    sds_o = None
    

In [4]:
dts_regexp = re.compile(r'.+A(\d{7}).+')
fls = glob.glob('/data/h5py_test/*hdf')

In [5]:
dts = [re.findall(dts_regexp,x)[0] for x in fls]
    
fls = [x for (y,x) in sorted(zip(dts,fls))]
    
dts.sort()

In [23]:
arr = init_shared(240*240*len(dts))

In [24]:
t1 = time.time() 
with closing(mp.Pool(processes=8,initializer = init, initargs = (arr,(240*240,len(dts)),fls))) as pool:
    
    res = pool.map(wload,[(0,0,2400,24,fix) for fix in range(0,len(fls))])

pool.close()
pool.join()
print(time.time() - t1)

21.242249488830566


In [9]:
arr2 = tonumpyarray(arr)


In [14]:
arr2 = arr2.reshape(24,2400,len(dts))

In [21]:
t1 = time.time() 
for fix in range(len(fls)):
    ds = gdal.Open(fls[fix])
    sds_o = gdal.Open(ds.GetSubDatasets()[0][0]) 
    
    arr2[...,fix] = sds_o.ReadAsArray(0,0,2400,24)
    
    ds = None
    sds_o = None
print(time.time() - t1)

96.19797039031982
